## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-10-05-45-45 +0000,bbc,'It's insulting they think we can't handle it'...,https://www.bbc.com/news/articles/ckg9e1nl40wo...
1,2025-12-10-05-41-55 +0000,nypost,Accused Georgia high school shooter Colt Gray ...,https://nypost.com/2025/12/10/us-news/georgia-...
2,2025-12-10-05-13-58 +0000,bbc,Why has the price of silver hit a record high?,https://www.bbc.com/news/articles/c62vn22523xo...
3,2025-12-10-05-06-53 +0000,bbc,Sperm from donor with cancer-causing gene was ...,https://www.bbc.com/news/articles/ckgmy90z991o...
4,2025-12-10-05-05-52 +0000,bbc,Kids locked out of social media apps after Aus...,https://www.bbc.com/news/articles/cgl6gkd7pz6o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2377/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
67,trump,39
295,up,14
123,new,13
38,out,13
456,ai,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
297,2025-12-09-11-56-07 +0000,nypost,"Trump warns Ukraine is ‘losing’ Russia war, ca...",https://nypost.com/2025/12/09/us-news/trump-sa...,88
57,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...,86
48,2025-12-10-02-08-00 +0000,wsj,President Trump touted the economy as he embar...,https://www.wsj.com/politics/policy/trump-tout...,82
74,2025-12-10-00-58-36 +0000,nyt,Trump Calls Europe ‘Decaying’ and Suggests ‘Si...,https://www.nytimes.com/2025/12/09/us/politics...,80
298,2025-12-09-11-54-00 +0000,wsj,President Trump now has enormous power over th...,https://www.wsj.com/politics/policy/in-warner-...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
297,88,2025-12-09-11-56-07 +0000,nypost,"Trump warns Ukraine is ‘losing’ Russia war, ca...",https://nypost.com/2025/12/09/us-news/trump-sa...
187,63,2025-12-09-19-12-00 +0000,wsj,"Bruce Blakeman, a Long Island Republican who t...",https://www.wsj.com/politics/elections/long-is...
65,55,2025-12-10-01-14-00 +0000,wsj,Kevin Hassett says he would rely on his own ju...,https://www.wsj.com/economy/central-banking/ke...
4,53,2025-12-10-05-05-52 +0000,bbc,Kids locked out of social media apps after Aus...,https://www.bbc.com/news/articles/cgl6gkd7pz6o...
303,43,2025-12-09-11-00-43 +0000,wapo,Have you seen this (AI-generated) man? Police ...,https://www.washingtonpost.com/nation/2025/12/...
283,42,2025-12-09-13-00-00 +0000,wsj,The Bank of Canada is expected to hold steady ...,https://www.wsj.com/articles/bank-of-canada-ex...
1,42,2025-12-10-05-41-55 +0000,nypost,Accused Georgia high school shooter Colt Gray ...,https://nypost.com/2025/12/10/us-news/georgia-...
298,38,2025-12-09-11-54-00 +0000,wsj,President Trump now has enormous power over th...,https://www.wsj.com/politics/policy/in-warner-...
286,34,2025-12-09-12-41-57 +0000,bbc,Congress ups pressure to release boat strike v...,https://www.bbc.com/news/articles/c773de38p2go...
296,34,2025-12-09-12-00-00 +0000,startribune,Minneapolis City Council looks to solve No. 1 ...,https://www.startribune.com/minneapolis-consid...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
